In [1]:
from KeRLas import Brain, TimedGymEnv, Trainer
from KeRLas.models import defaultQModel
from KeRLas.models import DirectDiffModel, NaiveQModel, LateralDiffModel, DQN
from KeRLas.policies import BoltzmannQPolicy
import numpy as np
import gym, math
from lunar_lander import LunarLander

#env = GymEnv(CartPoleEnv(), tlimit=300)
env = TimedGymEnv(LunarLander(), tlimit=500)

observation_width = env.observation_space.shape[0]
nactions = env.action_space.n


#for i in range(10):
#    next(brain.Memory.generate_samples(20))
#_ = next(brain.trainig_data_generator(mbsize))



Using TensorFlow backend.


WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.


In [2]:
rlmodel = DirectDiffModel(defaultQModel(observation_width, nactions), 0.99)
policy = BoltzmannQPolicy(0.0001)
brain = Brain(rlmodel, policy, training_policies=map(BoltzmannQPolicy, [10.0, 5.0, 2.0, 1.0, 0.01, 0.001]))


In [4]:
class Callback:
    
    def onEpisodeBegin(self, env, agents, observations):
        env.render()

    def onStep(self, env, env_done, feedback):
        env.render()
        
    def onEpisodeEnd(self, env, record):
        env.render()

trainer = Trainer(env, brain, 0.00, 10000)   

from KeRLas import GymPlayer
p = GymPlayer(env, brain, callback=Callback())

mbsize = 50

for i in xrange(50):
    metrics = trainer.train(mbsize, 1000, 50)
    
    nsum = 0
    N = 5
    reward_sum = 0.0
    with brain.training(False):
        for _ in range(N):
            history = p.runEpisode()
            reward = sum([r for _,_,_,r,_ in history])
            print reward, history[-1][3]
            reward_sum += reward
            #print len(history)
            nsum += len(history)
        print float(nsum)/N, reward_sum/N, metrics

-1.2767009003528695 -1.0
-1.4913096523289064 -1.0
-1.4417177821099978 -1.0
-1.340424027082054 -1.0
-1.3595852242930346 -1.0
65.8 -1.3819475172333724 4.722642e-05
-1.4126684499631723 -1.0
-1.425998392857547 -1.0
-1.3398491012284781 -1.0
-1.2991475562574624 -1.0
-1.3633271760940704 -1.0
74.0 -1.3681981352801462 2.0802947e-05
-1.524402606384041 -1.0
-1.6402932437537339 -1.0
-1.319105889429438 -1.0
-1.5079096904696312 -1.0
-1.459643893629263 -1.0
212.4 -1.4902710647332214 1.37258185e-05
-1.455524681243225 -1.0
-1.3964433221872712 -1.0
-1.419472933888818 -1.0
-1.451266843204654 -1.0
-1.3977842503080256 -1.0
74.2 -1.4240984061663986 6.115221e-05
-1.3883885815398713 -1.0
-1.379076717879309 -1.0
-1.7093677179182145 -1.0
-1.6627689452689587 -1.0
-1.3873301048554667 -1.0
67.8 -1.505386413492364 0.0002174257
-1.1018193678651738 -1.0
-0.9126838767343165 -1.0
-1.025357702837618 -1.0
-0.9874321976099258 -1.0
-1.0510441094281942 -1.0
70.8 -1.0156674508950456 2.9692294e-05
-1.0907357102032478 -1.0
-1.